In [33]:
from tensorboard.backend.event_processing import event_accumulator
import pandas as pd
from pathlib import Path


In [34]:
from tbparse import SummaryReader


def get_metric_results_from_eventfile(dirpath, metrics, save_path="", title=""):
    reader = SummaryReader(dirpath, pivot=False, extra_columns={'dir_name'})
    df = reader.scalars
    df = df.rename(columns={"tag": "metric", "dir_name": "fold"})
    df = df.groupby(["fold","metric"])['value'].max().reset_index()
    df = df[df["metric"].isin(metrics)].set_index(["fold"]).sort_values("metric").reset_index()
    res = df.pivot(index='fold', columns='metric', values='value').rename_axis(columns=None)
    if save_path:
        res.to_markdown(save_path, mode="a", tablefmt="github", index=True,floatfmt='.3f')
    return res

In [35]:
labels = ['Rote_X', 'Rote_Y', "Rote_Z",
          'Inference_X', "Inference_Y","Inference_Z",
          "Deep_X", "Deep_Z",
          "MW"]

In [36]:
metrics = ["train_auroc", "val_auroc", "train_accuracy_epoch", "val_accuracy_epoch"]
save_path = "../results/comprehension_results.md"
val_aurocs = []
for label in labels:
    tit=f"# {label}"
    with open(save_path, "a") as f:
        f.write("\n\n")
        f.write(tit)
        f.write("\n")
    res=get_metric_results_from_eventfile(f"../lightning_logs/from_alpine/informer_{label}", metrics, save_path)
    res['label']=label
    val_aurocs.append({"label":label, 
                       "mean AUROC": res[['val_auroc']].mean().item(),
                       "min AUROC": res[['val_auroc']].min().item(),
                       "max AUROC": res[['val_auroc']].max().item()
                       })

In [37]:
tit="Held-out AUROCs"
with open(save_path, "a") as f:
    f.write("\n\n")
    f.write(tit)
    f.write("\n")
tb = pd.DataFrame.from_records(val_aurocs)
tb.to_markdown(save_path, mode="a", tablefmt="github", index=False,floatfmt='.3f')
